# Seção 1: Inicialização e informações da GPU

In [1]:
!pip install git+https://github.com/canesche/nvcc4jupyter.git
!git clone https://github.com/canesche/nvcc4jupyter
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/canesche/nvcc4jupyter.git to /tmp/pip-req-build-kuofxpmf
  Running command git clone -q https://github.com/canesche/nvcc4jupyter.git /tmp/pip-req-build-kuofxpmf
  Created wheel for ColabPlugin: filename=ColabPlugin-blind-py3-none-any.whl size=12725 sha256=1c5ada0441a3abc84f32f29fcc490d1aa1eab7e0397e621b442af1dfea8129ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-y1tge0eg/wheels/8b/84/1d/a2cfacb702e232d99d06524968290563e63fdb70a4c78ee5c4
Failed to build ColabPlugin
    Running setup.py install for ColabPlugin ... done
  DEPRECATION: ColabPlugin was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 1

In [2]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [3]:
%%gpu
#include <cuda.h>
#include <stdio.h>

int main(){
    int dev = 0;
    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, dev);

    printf("Using Device %d:         %s\n",     dev, deviceProp.name);
    printf("Total global memory:     %u B\n",   deviceProp.totalGlobalMem);
    printf("Threads per block:       %d\n",     deviceProp.maxThreadsPerBlock);
    printf("Maximum Grid Size:       %d,%d,%d\n",deviceProp.maxGridSize[0],
                                                deviceProp.maxGridSize[1],
                                                deviceProp.maxGridSize[2]);
    printf("Maximum block size:      %d,%d,%d\n",deviceProp.maxThreadsDim[0],
                                                deviceProp.maxThreadsDim[1],
                                                deviceProp.maxThreadsDim[2]);
    return 0;
}

Using Device 0:         Tesla T4
Total global memory:     2958819328 B
Threads per block:       1024
Maximum Grid Size:       2147483647,65535,65535
Maximum block size:      1024,1024,64



# Exercício

No jogo campo minado, diversas bombas são distribuídas sobre um campo representado por uma matriz, e cada célula sem bomba guarda o número de bombas existentes nas 8 células ao seu redor. Dessa forma, o campo abaixo à esquerda,
onde cada bomba é representada por um *, pode ser armazenado pela matriz de inteiros mostrada à direita, onde cada bomba é representada por um 9. 
```
1 * 2 2 *                1 9 2 2 9 
2 4 * 3 1                2 4 9 3 1
* 5 * 2                  9 5 9 2 0
* * 3 1                  9 9 3 1 0
3 * 2                    3 9 2 0 0
```
Faça um programa que distribui de forma aleatória um número de bombas sobre um campo grande e, em seguida utiliza a GPU para calcular, de forma paralela, o número de bombas próximas a cada célula sem bomba.

Depois que estiver certo que seu kernel funciona, implemente também uma versão sequencial para fazer o mesma cálculo e insira marcadores de tempo para calcular e exibir o tempo gasto em cada uma das duas abordagens. Aumente o tamanho do campo progressivamente para valores muito grandes (ex: 10000x10000) e perceba que o tempo de cálculo na versão paralela tem um aumento relativamente baixo em relação à versão sequencial.

In [5]:
%%gpu

#include <stdio.h>
#include <cuda.h>
#include <stdexcept>
#include <chrono>

#define BLOCK_SIZE 32

/* A função CHECK a seguir pode ser utilizada caso seja necessário identificar
erros em chamadas da biblioteca CUDA. Exemplo: 
CHECK(cudaMalloc((void**)&x,size));
*/
#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
    }                                                                          \
}

__global__
void computeMatrixPar(int8_t *da, int rows, int cols) {
    int col=blockIdx.x * blockDim.x + threadIdx.x;
    int row=blockIdx.y * blockDim.y + threadIdx.y;

    if (col < cols && row < rows) {
        if (da[row*cols+col]!=9) {
            int n=0;
            for (int i=row-1; i<=row+1; ++i) {
                for (int j=col-1; j<=col+1; ++j) {
                    if (i>=0 && i<rows && j>=0 && j<cols && da[i*cols+j]==9)
                        n+=1;
                }
            }
            da[row*cols+col]=n;
        }
    }
}

void computeMatrixSeq(int8_t *a, int rows, int cols) {
    for (int row=0; row<rows; ++row)
        for (int col=0; col<cols; ++col)
            if (a[row*cols+col]!=9) {
                int n=0;
                for (int i=row-1; i<=row+1; ++i) {
                    for (int j=col-1; j<=col+1; ++j) {
                        if (i>=0 && i<rows && j>=0 && j<cols && a[i*cols+j]==9)
                            n+=1;
                    }
                }
                a[row*cols+col]=n;
            }
}

void distribuiBombas(int8_t *a, int linhas, int colunas, int num){
    if (num > linhas*colunas)
        throw std::invalid_argument("Número de bombas extrapolou o máximo possível.");

    for (int i=0; i<linhas; ++i)
        for (int j=0; j<colunas; ++j)
            a[i*colunas+j]=0;

    int lin, col;
    for (int i=0; i<num; ++i) {
        lin=rand()%linhas;
        col=rand()%colunas;
        while (a[lin*colunas+col] == 9) {
            lin=rand()%linhas;
            col=rand()%colunas;
        }
        a[lin*colunas+col]=9;
    }
}

void imprimeMatriz(int8_t *a, int linhas, int colunas ){
    int v;
    for (int i=0; i<linhas; ++i) {
        for (int j=0; j<colunas; ++j) {
            v=a[i*colunas+j];
            if (v==9) printf("* ");
            else if (v==0) printf("  ");
            else printf("%d ",v);
        }
        printf("\n");
    }
    printf("\n");
}

int main() {
    int tamanho=10000;
    int linhas=tamanho,colunas=tamanho;
    int nbombas=linhas*colunas/3;
    int size=linhas*colunas*sizeof(int8_t);
    int8_t *a, *b;
    int8_t *da;

    a=(int8_t*)malloc(size);
    b=(int8_t*)malloc(size);

    distribuiBombas(a,linhas,colunas,nbombas);
    //imprimeMatriz(a,linhas,colunas);

    //#Paralelo
    float tPar,tSeq;
    std::chrono::time_point<std::chrono::high_resolution_clock> tp1,tp2;
    tp1=std::chrono::high_resolution_clock::now();

    cudaMalloc((void**)&da,size);
    cudaMemcpy(da,a,size,cudaMemcpyHostToDevice);
    dim3 grid(ceil(1.0*colunas/BLOCK_SIZE), ceil(1.0*linhas/BLOCK_SIZE), 1);
    dim3 block(BLOCK_SIZE, BLOCK_SIZE, 1);
    computeMatrixPar<<<grid, block>>>(da,linhas,colunas);
    cudaMemcpy(b,da,size,cudaMemcpyDeviceToHost);
    cudaFree(da);
    cudaDeviceReset();

    tp2=std::chrono::high_resolution_clock::now();
    tPar=std::chrono::duration<double,std::ratio<1,1000>> (tp2-tp1).count();

    //#Sequencial
    tp1=std::chrono::high_resolution_clock::now();

    computeMatrixSeq(a,linhas,colunas);

    tp2=std::chrono::high_resolution_clock::now();
    tSeq=std::chrono::duration<double,std::ratio<1,1000>> (tp2-tp1).count();

    //#Resultados
    //imprimeMatriz(a,linhas,colunas);
    //imprimeMatriz(b,linhas,colunas);

    printf("Tempo paralelo: %.3fms\nTempo sequencial: %.3fms\n",tPar,tSeq);

    free(a);
    free(b);
    return 0;
}


Tempo paralelo: 900.197ms
Tempo sequencial: 3936.016ms

